# TripPal Trips Planner

In [2]:
import numpy as np 
import pandas as pd
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from pathlib import Path

module_path = str(Path.cwd().parents[0] / "SearchEngine")

if module_path not in sys.path:
    sys.path.append(module_path)
    
from Search_engine import Search_Engine 

ModuleNotFoundError: No module named 'SearchEngine'

## 1. Fetching data

In [ ]:
search_engine = Search_Engine()

In [ ]:
trip = search_engine.collect_trip_components('milan', 3, ['CUL'], 'extended_trip', True, True)

In [ ]:
trip = search_engine.collect_trip_components('milan', 3, ['CUL'], 'extended_trip', False, True)

In [3]:
with open('samples/test_trip_data.pkl', 'rb') as inp:
    m_trip = pickle.load(inp)

In [33]:
with open('test_trip_data.pkl', 'wb') as output:
    pickle.dump(list(trip), output, pickle.HIGHEST_PROTOCOL)

In [28]:
with open('test_trip_data.pkl', 'rb') as input:
    l_trip = pickle.load(input)

In [30]:
l_trip

[- Type: cultural, ID: R7828254, Rate: 7, Coordinates: {'lat': 45.47237, 'lon': 9.173939},
 - Type: cultural, ID: N4754691506, Rate: 7, Coordinates: {'lat': 45.478031, 'lon': 9.122564},
 - Type: food, ID: N5759888769, Rate: 1, Coordinates: {'lat': 45.480495, 'lon': 9.146198},
 - Type: hotel, ID: 194687, Rate: 8.6, Coordinates: {'lat': 45.47892, 'lon': 9.14523},
 - Type: shop, ID: W166623693, Rate: 7, Coordinates: {'lat': 45.465611, 'lon': 9.190014},
 - Type: food, ID: N1789640322, Rate: 1, Coordinates: {'lat': 45.478992, 'lon': 9.142301},
 - Type: food, ID: N5759888798, Rate: 1, Coordinates: {'lat': 45.479523, 'lon': 9.14592},
 - Type: food, ID: N5759888485, Rate: 1, Coordinates: {'lat': 45.480743, 'lon': 9.143989},
 - Type: shop, ID: N4714788078, Rate: 7, Coordinates: {'lat': 45.465008, 'lon': 9.18996},
 - Type: shop, ID: W331883966, Rate: 7, Coordinates: {'lat': 45.465611, 'lon': 9.190014},
 - Type: food, ID: N2870835155, Rate: 1, Coordinates: {'lat': 45.479721, 'lon': 9.146256},
 - 

In [4]:
with open('samples/milan_trip_data.pkl','rb') as inp:
    m_trip = pickle.load(inp)
m_trip

{- Type: cafes, ID: N26065697, Rate: 3, Coordinates: {'lat': 40.428734, 'lon': -3.702002},
 - Type: cafes, ID: N3426617375, Rate: 3, Coordinates: {'lat': 40.422195, 'lon': -3.692342},
 - Type: cafes, ID: Q641473, Rate: 3, Coordinates: {'lat': 40.41798, 'lon': -3.699533},
 - Type: cultural, ID: N255288522, Rate: 7, Coordinates: {'lat': 40.423428, 'lon': -3.68889},
 - Type: cultural, ID: N936505939, Rate: 7, Coordinates: {'lat': 40.432259, 'lon': -3.69773},
 - Type: cultural, ID: W335583298, Rate: 7, Coordinates: {'lat': 40.422863, 'lon': -3.692843},
 - Type: fast_food, ID: N1128664813, Rate: 1, Coordinates: {'lat': 40.427082, 'lon': -3.694992},
 - Type: fast_food, ID: N1316546599, Rate: 1, Coordinates: {'lat': 40.426128, 'lon': -3.692271},
 - Type: fast_food, ID: N1316546986, Rate: 1, Coordinates: {'lat': 40.426636, 'lon': -3.693338},
 - Type: historic, ID: N5033133939, Rate: 7, Coordinates: {'lat': 40.424965, 'lon': -3.69284},
 - Type: historic, ID: Q28503050, Rate: 7, Coordinates: {'l

## 2. Filtering data

## 3. Preview trip data

In [3]:

module_path = str(Path.cwd().parents[0] / "SearchEngine")

if module_path not in sys.path:
    sys.path.append(module_path)
    
from trip_classes.Item import Item 
from trip_classes.Day import Day 

ModuleNotFoundError: No module named 'Item'

In [2]:

# from Item import Item
import pickle
from icecream import ic
import random

from math import floor
from haversine import haversine

In [42]:

items = list(m_trip)
cities_names = []


def get_distance(item1: Item, item2: Item):
    cord1 = item1.coordinate
    cord2 = item2.coordinate
    tuple1 = (cord1['lat'], cord1['lon'])
    tuple2 = (cord2['lat'], cord2['lon'])
    return haversine(tuple1, tuple2)


class Planner:
    optimal_route = []
    optimal_cost = 0
    constraints = {
        'malls': 1,
        'restaurants': 2,
        'cafes': 3,
        'fast_food': 4,
        'architecture': 5,
        'cultural': 6,
        'sport': 7,
        'natural': 8,
        'marketplaces': 9,
        'hotel': 1000
    }

    def __init__(self, items):
        self.graph = []
        # self.G = nx.Graph()
        # self.P = nx.Graph()
        for i in range(len(items)):
            cities_names.append(f"{items[i].item_id}, Type:{items[i].item_type}")
            self.graph.append([])
            for j in range(len(items)):
                self.graph[i].append(floor(get_distance(items[i], items[j]) * 1000) / 1000)

    def delta(self, n1, n2, n3, n4):
        return self.graph[n1][n3] + self.graph[n2][n4] - self.graph[n1][n2] - self.graph[n3][n4]

    def plan_two_opt(self, iterations=5):
        i = 0
        total_costs = []
        while i < iterations:
            i += 1
            initial_route = [0] + random.sample(range(1, len(cities_names)), len(cities_names) - 1)

            best_route = initial_route
            improved = True
            while improved:
                improved = False

                for i in range(1, len(self.graph) - 2):
                    for j in range(i + 1, len(self.graph)):
                        if j - i == 1:
                            continue
                        if self.delta(best_route[i - 1], best_route[i], best_route[j - 1], best_route[j]) < 0:
                            best_route[i:j] = best_route[j - 1:i - 1:-1]
                            improved = True
            path = [items[i] for i in best_route]
#             [self.G.add_edge(items[i - 1], items[i]) for i in best_route]
            cost = 0
            for i in range(1, len(path) - 1):
                cost += get_distance(path[i], path[i - 1])

                total_costs.append((cost, best_route))
        total_costs = sorted(total_costs, key=lambda x: x[0])
        self.optimal_cost, self.optimal_route = total_costs[0]
        return self.optimal_route, self.optimal_cost, path

    def get_region_boundary(self, item):
        R = 6378.1  # Radius of the Earth
        brng = 1.57  # Bearing is 90 degrees converted to radians.
        d = 50  # Distance in km
        lat1, lon1 = math.radians(item.coordinate['lat']), math.radians(item.coordinate['lon'])
        # lat1 = math.radians(52.20472)  # Current lat point converted to radians
        # lon1 = math.radians(0.14056)  # Current long point converted to radians

        lat2 = math.asin(math.sin(lat1) * math.cos(d / R) +
                         math.cos(lat1) * math.sin(d / R) * math.cos(brng))

        lon2 = lon1 + math.atan2(math.sin(brng) * math.sin(d / R) * math.cos(lat1),
                                 math.cos(d / R) - math.sin(lat1) * math.sin(lat2))

        lat2 = floor(math.degrees(lat2) * 1000) / 1000
        lon2 = floor(math.degrees(lon2) * 1000) / 1000
        return lat2, lon2


def get_sublists(original_list, number_of_sub_list_wanted):
    sublists = list()
    for sub_list_count in range(number_of_sub_list_wanted):
        sublists.append(original_list[sub_list_count::number_of_sub_list_wanted])
    return sublists


if __name__ == '__main__':
    planner = Planner(items)
    optimal_route, optimal_cost, path = planner.plan_two_opt(iterations=1)
    ic(optimal_cost)

    # nx.draw(planner.G, node_color=['red'] * (len(path)), with_labels=True)
    # plt.show()
    range = []
    sub_region = []
    cpath = path.copy()
    while cpath:
        src = cpath[0]
        for p in path:
            if p in cpath:
                dist = get_distance(src, p)
                if dist < 50:
                    range.append((p, dist))
                    cpath.remove(p)
        sub_region.append(range)
        range = []
    days = []
    for sub in sub_region:
        a = []
        if len(sub) > 4:
            for s in sub:
                a.append(s)
                if len(a) >= 5:
                    days.append(Day(len(days), a))
                    a = []
        else:
            days.append(Day(len(days), sub))
    ic(days)


ic| optimal_cost: 0.29087107321172834


NameError: name 'Day' is not defined

In [2]:

import pickle
with open('samples/milan_trip_data.pkl', 'rb') as input:
    l_trip = pickle.load(input)


ModuleNotFoundError: No module named 'search_engine'